In [1]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate, ShuffleSplit, train_test_split
import pandas as pd
import data_pecarn 
import data_psrc
import data
import util
from style import *
from collections import Counter
from data import feats_numerical, feats_categorical, meta, outcome_def

# plt.style.use('dark_background')
# sns.set(style="white")
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [77]:
d = data_pecarn.get_data(use_processed=False,
                                 frac_missing_allowed=0.1,
                                 dummy=True,
                                 impute_feats=False)

computing pecarn preprocessing...


# benchmark bivariable associations

In [78]:
all_feats_pecarn, filtered_feats_pecarn = data.get_feat_names(d)

In [86]:
d['Age<2'] = d['Age'] < 2
d['GCS<14'] = d['GCSScore'] < 14
data.select_final_feats(filtered_feats_pecarn)

['AbdDistention_yes',
 'AbdTenderDegree_Mild',
 'AbdTenderDegree_Moderate',
 'AbdTenderDegree_None',
 'AbdTenderDegree_Severe',
 'AbdTrauma_or_SeatBeltSign_yes',
 'AbdomenPain_yes',
 'Age',
 'Age<2',
 'CostalTender',
 'DecrBreathSound_yes',
 'DistractingPain_yes',
 'GCSScore_Full_yes',
 'Hypotension_yes',
 'InitHeartRate',
 'InitSysBPRange',
 'MOI_Bike collision/fall',
 'MOI_Fall down stairs',
 'MOI_Fall from an elevation',
 'MOI_Motor vehicle collision',
 'MOI_Motorcycle/ATV/Scooter collision',
 'MOI_Object struck abdomen',
 'MOI_Pedestrian/bicyclist struck by moving vehicle',
 'Sex_F',
 'Sex_M',
 'ThoracicTender_yes',
 'ThoracicTrauma_yes',
 'VomitWretch_yes']

In [93]:
ks = ['AbdomenPain_yes', 'AbdTrauma_or_SeatBeltSign_yes', 'GCS<14',
      'AbdTenderDegree_None', 'AbdTenderDegree_Mild',
      'AbdTenderDegree_Moderate', 'AbdTenderDegree_Severe',
      'ThoracicTrauma_yes',
      'DecrBreathSound_yes',
      'VomitWretch_yes',
      'Age<2',
      'MOI_Motor vehicle collision',
      'Hypotension_yes',
       'ThoracicTender_yes',
 'CostalTender',
'AbdDistention_yes',
 'DistractingPain_yes'
     ]
index = ['NA', 'IAIw/I', 'No IAIw/I', 'Difference']
o = data.outcome_def
r = {k: [] for k in ks}
io = d[o] == 1
# print(d[io])
for k in ks:
    ik = d[k] == 1 # d[o]==1
    inan = d[k].isna()
    r[k].append(np.sum(inan))   
    num1 = np.sum(d[ik][o]==1)
    denom1 = (io & ~inan).sum()
    r[k].append(f'{num1}/{denom1}')
    num2 = np.sum(d[ik][o]==0)
    denom2 = (~io & ~inan).sum()
    r[k].append(f'{num2}/{denom2}')
    r[k].append(f'{num1/denom1 - num2/denom2:0.3f}')    
r = pd.DataFrame.from_dict(r)
r.index = index
r.transpose()

,NA,IAIw/I,No IAIw/I,Difference
AbdomenPain_yes,0,105/203,3515/11841,0.220
AbdTrauma_or_SeatBeltSign_yes,0,112/203,1851/11841,0.395
GCS<14,0,65/203,902/11841,0.244
AbdTenderDegree_None,0,90/203,8357/11841,-0.262
AbdTenderDegree_Mild,0,12/203,1623/11841,-0.078
AbdTenderDegree_Moderate,0,42/203,1501/11841,0.080
AbdTenderDegree_Severe,0,59/203,360/11841,0.260
ThoracicTrauma_yes,0,66/203,1976/11841,0.158
DecrBreathSound_yes,0,25/203,231/11841,0.104
VomitWretch_yes,0,49/203,1104/11841,0.148


# recreate / evaluate rule

In [16]:
def pecarn_rule_predict(d, o=data.outcome_def):
    print(f'{d[o].sum()} / {d.shape[0]}')
    risks = np.array([np.nan] * d.shape[0])
    
    idxs = d['AbdTrauma_or_SeatBeltSign_yes']==1
    do = d[idxs]
    d = d[~idxs]
    print(f'{d[o].sum()} / {d.shape[0]}\t{do[o].sum()} / {do.shape[0]}')
    
    idxs = (d['GCSScore']<14) & (d['GCSScore'] > 2)
    do = d[idxs]
    d = d[~idxs]
    print(f'{d[o].sum()} / {d.shape[0]}\t{do[o].sum()} / {do.shape[0]}')


pecarn_rule_predict(d)

203 / 12044
91 / 10081	112 / 1963
55 / 9279	36 / 802


In [42]:
d['AbdTrauma_or_SeatBeltSign_yes'].unique()

array([0, 1], dtype=uint64)